In [ ]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.

#  FROM : https://www.tensorflow.org/tutorials/layers#building_the_cnn_mnist_classifier
#  CODE : https://www.tensorflow.org/code/tensorflow/examples/tutorials/layers/cnn_mnist.py

"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features, [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)
  #logits = tf.layers.dense(inputs=dense, units=10)

  loss = None
  train_op = None

  # Calculate Loss (for both TRAIN and EVAL modes)
  if mode != learn.ModeKeys.INFER:
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == learn.ModeKeys.TRAIN:
    train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=0.001,
        #optimizer="SGD")
        optimizer="Adam")

  # Generate Predictions
  predictions = {
      "classes": tf.argmax(
          input=logits, axis=1),
      "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor"), 
      "logits": logits,
  }

  # Return a ModelFnOps object
  return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [ ]:
# Load training and eval data
mnist = learn.datasets.load_dataset("mnist")

train_data   = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)

eval_data    = mnist.test.images  # Returns np.array
eval_labels  = np.asarray(mnist.test.labels, dtype=np.int32)

In [ ]:
# Create the Estimator : https://www.tensorflow.org/extend/estimators
import os
this_dir = os.getcwd()

mnist_classifier = learn.Estimator(
      model_fn=cnn_model_fn, model_dir="mnist_convnet_model")  # This is relative to the ipynb
      #model_fn=cnn_model_fn, model_dir=os.path.join(this_dir, "mnist_convnet_model"))  # This is system-absolute

# NB :  Check the checkpoints file in 'mnist_convnet_model has filenames that are in same directory

In [ ]:
# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook( tensors=tensors_to_log, every_n_iter=50 )

# Train the model
mnist_classifier.fit(
  x=train_data,
  y=train_labels,
  batch_size=100,
  steps=50000/100 * 5,
#  steps=20000,
#  monitors=[logging_hook]
)

In [ ]:
#mnist_classifier.save()

In [ ]:
# Configure the accuracy metric for evaluation
metrics = {
  "accuracy":
      #learn.metric_spec.MetricSpec(
      learn.MetricSpec(
          metric_fn=tf.metrics.accuracy, prediction_key="classes"),
}

# Evaluate the model and print results
eval_results = mnist_classifier.evaluate( x=eval_data, y=eval_labels, metrics=metrics)

print(eval_results)

Ok, so the built Estimator gets 99.26% accuracy on the test set.

### Next Steps

Let's :

*  go through the training set and store the logits for [the valid?] ALL the numbers;

*  build an AutoEncoder on the logits, which minimises reconstruction error;

*  histogram the reconstruction error to find a bound above which we can reject an input image;

*  create adversarial examples that fool the original Estimator;

*  check that the adversarial examples score outside the bound;

*  attempt to create adversarial examples on an updated network that includes the autoencoder bound as a gating function on the rest of the outputs;

*  create an infoGAN network for MNIST that allows us to create digits that are 'between' two classes;

*  score the reconstruction error of the between images to look at the rejection regions (which hopefully isolate the islands of acceptance from one another)




#### Get logit representation for all training examples


In [ ]:
# Evaluate the model and gather the results
predictions = mnist_classifier.predict( x=train_data, batch_size=100, as_iterable=True)

#for p in predictions:
#    print(p)
#    break

train_data_logits = np.array( [ p['logits'] for p in predictions ])
train_data_logits.shape

In [ ]:
# So, let's build an autoencoder 'regression' model with a hidden layer with input/2 units 

def autoencoder_model_fn(features, labels, mode):
  #print("features.shape", features.get_shape())
  #dim = features.get_shape()[1]
  dim = 10

  # Input Layer
  input_layer = tf.reshape(features, [-1, dim])

  # Encoder Dense Layer
  # Densely connected layer with size/2 neurons
  # Input Tensor Shape: [batch_size, 10]
  # Output Tensor Shape: [batch_size, 10/2]
  dense = tf.layers.dense(inputs=input_layer, units=dim, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  #dropout = tf.layers.dropout(
  #    inputs=dense, rate=0.9, training=mode == learn.ModeKeys.TRAIN)

  # Decoder Dense Layer
  # Input Tensor Shape: [batch_size, 10/2]
  # Output Tensor Shape: [batch_size, 10]
  #output_layer = tf.layers.dense(inputs=dropout, units=dim)
  output_layer = tf.layers.dense(inputs=dense, units=dim)  # Linear activation

  loss = None
  train_op = None

  ## Calculate Loss (for both TRAIN and EVAL modes)
  #if mode != learn.ModeKeys.INFER:
  #  loss = tf.losses.mean_squared_error( input_layer, output_layer )

  loss = tf.losses.mean_squared_error( input_layer, output_layer )

  # Configure the Training Op (for TRAIN mode)
  if mode == learn.ModeKeys.TRAIN:
    train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=0.001,
        optimizer="Adam")

  # Generate Predictions
  predictions = {
      "rmse": loss,
  }

  # Return a ModelFnOps object
  return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [ ]:
mnist_autoencoder = learn.Estimator(
      model_fn=autoencoder_model_fn, model_dir="mnist_model/autoencoder")

In [ ]:
# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
#tensors_to_log = {"probabilities": "softmax_tensor"}
#logging_hook = tf.train.LoggingTensorHook( tensors=tensors_to_log, every_n_iter=50 )

# Train the model
mnist_autoencoder.fit(
  x=train_data_logits,
  y=train_data_logits,
  batch_size=100,
  steps=50000/100 * 50,
#  steps=20000,
#  monitors=[logging_hook]
)

*  INFO:tensorflow:Saving checkpoints for 25000 into mnist_model/autoencoder/model.ckpt.
*  INFO:tensorflow:Loss for final step: 1.2686.
